In [3]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm

### **[데이터 로드]**

In [4]:
# OpenAI text-embedding-3-small 통해 추출하였음
# summary + keywords 수평합

df_embed = pd.read_csv("../sia_embed.csv") 
df_llm = pd.read_csv("../summary_df_final.csv") 
df_raw = pd.read_csv("spnews_final.csv")

In [5]:
df_embed.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_3064,feature_3065,feature_3066,feature_3067,feature_3068,feature_3069,feature_3070,feature_3071,category,id
0,0.022873,0.057617,0.077078,0.025437,-0.039811,0.045100,0.055962,-0.013990,-0.022873,-0.035208,...,0.047363,0.010006,0.033351,0.037054,0.041374,-0.000171,-0.004075,-0.009012,경제/산업,spnews_101404
1,0.017431,0.007211,0.038271,0.017969,-0.002414,-0.053658,-0.003341,0.022324,0.009124,-0.002883,...,-0.001979,-0.048874,0.011022,-0.012274,0.005244,0.008429,0.001900,-0.010199,경제/산업,spnews_101403
2,0.035629,0.045025,0.052281,0.017052,-0.039927,0.021285,-0.000493,0.005451,-0.003000,-0.030978,...,0.024440,0.008236,0.011784,0.019709,0.029847,-0.003328,-0.026468,0.014008,사회/문화/체육,spnews_101402
3,0.014035,-0.002810,-0.015359,-0.015873,0.056814,0.003094,-0.002788,0.059026,-0.014697,-0.032259,...,0.016710,0.001253,0.010360,-0.015580,-0.006557,0.031748,0.018959,-0.046285,경제/산업,spnews_101394
4,0.025085,0.029958,-0.027741,-0.023427,-0.033613,-0.060715,0.019293,0.035191,-0.058438,-0.002541,...,-0.035220,-0.007421,0.009615,-0.013928,0.023839,0.019958,-0.022442,-0.028646,사회/문화/체육,spnews_101392


In [6]:
df_llm.head(5)

,id,summary,keywords,event_title,event_date,event_person,event_org,event_loc,p_rice(won/kg),p_corn(won/kg),p_usd(won/usd),job_cost
0,spnews_101404,신의주온실종합농장 건설장에서 지대정리와 잔디심기가 마감단계에 이르고 있다. 군민건설...,"신의주, 온실농장, 지대정리, 잔디심기, 건설",신의주온실종합농장 건설 마감단계,2025-11-17,NaN,NaN,"평안북도, 신의주",NaN,NaN,NaN,0.000318
1,spnews_101403,북한이 나무의 사름률을 높이는 과학기술제품으로 천연생물활성제를 소개했다. 이 제품은...,"나무, 사름률, 천연생물활성제, 제품, 북한","북한, 나무 사름률 높이는 제품 소개",2025-11-17,NaN,NaN,NaN,NaN,NaN,NaN,0.000335
2,spnews_101402,북한 개성시에서 식물방역소와 돼지종축장이 새로 건설되었다. 개성시식물방역소 건설에 ...,"개성시, 식물방역소, 돼지종축장, 건설, 농업","개성시, 식물방역소와 돼지종축장 신축",2025-11-17,NaN,"개성시식물방역소, 시남새온실, 시유기질복합비료공장",개성시,NaN,NaN,NaN,0.000313
3,spnews_101394,"가을 추수가 마무리되면서 북한의 곡물 가격이 하락세를 보이고 있다. 평양, 혜산, ...","곡물가, 하락, 가을 추수, 물가, 북한",가을 추수로 인한 곡물가 하락,2025-11-15,NaN,NaN,"평양시, 혜산시, 원산시, 사리원시",19850.0,4500.0,34600.0,0.000379
4,spnews_101392,북한 각지에서 어머니날을 경축하며 김일성-김정일 동상에 꽃다발을 바쳤다. 공산주의어...,"어머니날, 경축, 동상, 공연, 여성근로자",어머니날 경축 행사,2025-11-16,NaN,NaN,"평안남도, 황해북도, 강원도, 양강도, 나선시, 개성시",NaN,NaN,NaN,0.000316


In [7]:
df = df_embed.merge(df_raw[["id", "publish_date"]], on="id", how="left")

### **[기사 추천 코드]**

In [8]:
feature_cols = [col for col in df.columns if col.startswith("feature_")]
embeddings = df[feature_cols].values.astype(np.float32)
categories = df['category'].values 
publish_dates = df["publish_date"].values
ids = df['id'].values

In [9]:
norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
emb_norm = embeddings / norms

In [10]:
id_to_index = {str(article_id): idx for idx, article_id in enumerate(ids)}

In [ ]:
def get_similar_articles(click_id, k):
    """

    [작성 2025-11-25]
    click_id: string (ex. "spnews_101404")

    test 파일 기준이므로, id가 integer가 아닌 string. 실제 코드에서는 ID(integer) 활용.

    유저가 특정 기사 클릭 시, 관련 기사 k개(테스트는 10개)를 추천.

    [수정 2025-11-25]
    raw 데이터로부터 publish_date 가져와 함께 출력.
    실제 코드에서는 raw 와 llm date가 하나의 table에 있음 유의. 


    """
    click_id = str(click_id)

    if click_id not in id_to_index:
        raise ValueError(f"'{click_id}' 는 존재하지 않는 id 입니다. 다른 id로 시도해주세요.")

    click_idx = id_to_index[click_id]
    query_vec = emb_norm[click_idx]             

    sims = emb_norm @ query_vec                  
    sims[click_idx] = -1.0                      

    top_idx = np.argsort(-sims)[:k]

    results = []
    for idx in top_idx:
        results.append({
            "id": ids[idx],
            "category": categories[idx],
            "publish_date": str(publish_dates[idx])[:10],
            "similarity": float(sims[idx])
        })
    return results

### **[테스트]**

In [12]:
# test1
example_id = "spnews_101404"
similar_articles = get_similar_articles(example_id, k = 10)

In [13]:
print(f"유저 클릭 기사 id: {example_id}")    
print("관련 추천 기사 10개:")
for rank, item in enumerate(similar_articles, start=1):
    print(
        f"{rank:2d}. id={item['id']}, " 
        f"category={item['category']}, "
        f"publish_date={item['publish_date']}, "
        f"sim={item['similarity']:.4f}"
        )

유저 클릭 기사 id: spnews_101404
관련 추천 기사 10개:
 1. id=spnews_63973, category=경제/산업, publish_date=2023-04-10, sim=0.5319
 2. id=spnews_48611, category=정치, publish_date=2022-01-28, sim=0.5278
 3. id=spnews_55668, category=경제/산업, publish_date=2022-09-05, sim=0.5268
 4. id=spnews_100345, category=정치, publish_date=2025-10-18, sim=0.5181
 5. id=spnews_68176, category=경제/산업, publish_date=2023-07-18, sim=0.5142
 6. id=spnews_97311, category=경제/산업, publish_date=2025-07-17, sim=0.5087
 7. id=spnews_100387, category=사회/문화/체육, publish_date=2025-10-20, sim=0.5086
 8. id=spnews_50350, category=경제/산업, publish_date=2022-03-20, sim=0.5080
 9. id=spnews_101402, category=사회/문화/체육, publish_date=2025-11-17, sim=0.5059
10. id=spnews_52009, category=경제/산업, publish_date=2022-05-07, sim=0.5051


In [14]:
# test2
example_id = "spnews_100196"    
similar_articles = get_similar_articles(example_id, k = 10)

In [15]:
print(f"유저 클릭 기사 id: {example_id}")    
print("관련 추천 기사 10개:")
for rank, item in enumerate(similar_articles, start=1):
    print(
        f"{rank:2d}. id={item['id']}, " 
        f"category={item['category']}, "
        f"publish_date={item['publish_date']}, "
        f"sim={item['similarity']:.4f}"
        )

유저 클릭 기사 id: spnews_100196
관련 추천 기사 10개:
 1. id=spnews_21618, category=경제/산업, publish_date=2019-08-18, sim=0.5412
 2. id=spnews_93112, category=사회/문화/체육, publish_date=2025-03-14, sim=0.5069
 3. id=spnews_97526, category=사회/문화/체육, publish_date=2025-07-23, sim=0.4968
 4. id=spnews_45562, category=경제/산업, publish_date=2021-11-07, sim=0.4639
 5. id=spnews_42462, category=경제/산업, publish_date=2021-08-21, sim=0.4610
 6. id=spnews_25297, category=경제/산업, publish_date=2020-01-13, sim=0.4594
 7. id=spnews_23212, category=경제/산업, publish_date=2019-10-23, sim=0.4563
 8. id=spnews_95108, category=경제/산업, publish_date=2025-05-09, sim=0.4509
 9. id=spnews_93712, category=사회/문화/체육, publish_date=2025-03-28, sim=0.4459
10. id=spnews_31088, category=경제/산업, publish_date=2020-07-30, sim=0.4452


In [16]:
# test3
example_id = "spnews_99289"    
similar_articles = get_similar_articles(example_id, k = 10)

In [17]:
print(f"유저 클릭 기사 id: {example_id}")    
print("관련 추천 기사 10개:")
for rank, item in enumerate(similar_articles, start=1):
    print(
        f"{rank:2d}. id={item['id']}, " 
        f"category={item['category']}, "
        f"publish_date={item['publish_date']}, "
        f"sim={item['similarity']:.4f}"
        )

유저 클릭 기사 id: spnews_99289
관련 추천 기사 10개:
 1. id=spnews_5566, category=정치, publish_date=2017-12-02, sim=0.6166
 2. id=spnews_71160, category=외교, publish_date=2023-10-01, sim=0.6110
 3. id=spnews_25525, category=정치, publish_date=2020-01-22, sim=0.5934
 4. id=spnews_7831, category=정치, publish_date=2018-03-07, sim=0.5839
 5. id=spnews_48308, category=정치, publish_date=2022-01-20, sim=0.5715
 6. id=spnews_17763, category=정치, publish_date=2019-03-13, sim=0.5681
 7. id=spnews_63244, category=외교, publish_date=2023-03-22, sim=0.5670
 8. id=spnews_4404, category=정치, publish_date=2017-10-21, sim=0.5641
 9. id=spnews_10018, category=정치, publish_date=2018-05-24, sim=0.5617
10. id=spnews_80544, category=외교, publish_date=2024-05-27, sim=0.5615


In [18]:
df_llm.head(5)

,id,summary,keywords,event_title,event_date,event_person,event_org,event_loc,p_rice(won/kg),p_corn(won/kg),p_usd(won/usd),job_cost
0,spnews_101404,신의주온실종합농장 건설장에서 지대정리와 잔디심기가 마감단계에 이르고 있다. 군민건설...,"신의주, 온실농장, 지대정리, 잔디심기, 건설",신의주온실종합농장 건설 마감단계,2025-11-17,NaN,NaN,"평안북도, 신의주",NaN,NaN,NaN,0.000318
1,spnews_101403,북한이 나무의 사름률을 높이는 과학기술제품으로 천연생물활성제를 소개했다. 이 제품은...,"나무, 사름률, 천연생물활성제, 제품, 북한","북한, 나무 사름률 높이는 제품 소개",2025-11-17,NaN,NaN,NaN,NaN,NaN,NaN,0.000335
2,spnews_101402,북한 개성시에서 식물방역소와 돼지종축장이 새로 건설되었다. 개성시식물방역소 건설에 ...,"개성시, 식물방역소, 돼지종축장, 건설, 농업","개성시, 식물방역소와 돼지종축장 신축",2025-11-17,NaN,"개성시식물방역소, 시남새온실, 시유기질복합비료공장",개성시,NaN,NaN,NaN,0.000313
3,spnews_101394,"가을 추수가 마무리되면서 북한의 곡물 가격이 하락세를 보이고 있다. 평양, 혜산, ...","곡물가, 하락, 가을 추수, 물가, 북한",가을 추수로 인한 곡물가 하락,2025-11-15,NaN,NaN,"평양시, 혜산시, 원산시, 사리원시",19850.0,4500.0,34600.0,0.000379
4,spnews_101392,북한 각지에서 어머니날을 경축하며 김일성-김정일 동상에 꽃다발을 바쳤다. 공산주의어...,"어머니날, 경축, 동상, 공연, 여성근로자",어머니날 경축 행사,2025-11-16,NaN,NaN,"평안남도, 황해북도, 강원도, 양강도, 나선시, 개성시",NaN,NaN,NaN,0.000316


In [19]:
row1 = df_llm[df_llm["id"] == "spnews_42462"].iloc[0]
print(row1["summary"],row1["keywords"])

북한 황해남도 은률광산 대형 장거리 벨트콘베아가 건설된 지 46년이 지나면서 제방 주변에 숲이 형성되고 있다. 수십만 그루의 나무가 자라고 있으며, 40년 이상 자란 나무도 존재한다. 벨트콘베아는 연간 1,500만 톤의 버럭을 운반하고 있다. 벨트콘베아, 은률광산, 숲, 제방, 나무


In [20]:
row2 = df_llm[df_llm["id"] == "spnews_100196"].iloc[0]
print(row2["summary"],row2["keywords"])

북한이 백두산 관광객을 대비하여 양강도 삼지연에 베개봉호텔을 개관했다. 호텔은 현대적인 건축물과 자연 경관이 조화를 이루며, 스키장과의 근접성으로 스키 활동에 편리하다. 기존의 베개봉호텔은 철거되고 신축된 것으로 보인다. 베개봉호텔, 삼지연, 백두산, 관광, 스키
